In [1]:
import pandas as pd

In [15]:
train = pd.read_csv('../processed_data/train_v7.csv')
test = pd.read_csv('../processed_data/test_v7.csv')

In [6]:
train['location']

0              東京都北区滝野川３丁目
1              東京都中央区月島３丁目
2              東京都渋谷区笹塚２丁目
3        東京都杉並区高円寺南２丁目23-2
4           東京都葛飾区金町３丁目7-2
               ...        
31464          東京都板橋区蓮根２丁目
31465    東京都世田谷区太子堂５丁目17-1
31466          東京都江東区南砂４丁目
31467          東京都中野区中野２丁目
31468          東京都豊島区千川２丁目
Name: location, Length: 31469, dtype: object

In [14]:
train['location'].str.extract('(.+)都(.*)区(.*)',expand=True)

,0,1,2
0,東京,北,滝野川３丁目
1,東京,中央,月島３丁目
2,東京,渋谷,笹塚２丁目
3,東京,杉並,高円寺南２丁目23-2
4,東京,葛飾,金町３丁目7-2
...,...,...,...
31464,東京,板橋,蓮根２丁目
31465,東京,世田谷,太子堂５丁目17-1
31466,東京,江東,南砂４丁目
31467,東京,中野,中野２丁目


In [16]:
layout = pd.concat([train['layout'],test['layout']])

In [19]:
layout.value_counts()

1K            23661
1R            10177
1LDK           9702
2LDK           5131
1DK            3781
2DK            3715
3LDK           2240
2K             1807
3DK             851
1LDK+S(納戸)      487
2LDK+S(納戸)      326
4LDK            250
3LDK+S(納戸)      133
1K+S(納戸)        130
3K              102
2DK+S(納戸)        38
5LDK             38
4DK              33
1DK+S(納戸)        27
4LDK+S(納戸)       24
4K               12
3DK+S(納戸)        10
6LDK              9
5LDK+S(納戸)        9
5DK               8
5K                7
6LDK+S(納戸)        4
2K+S(納戸)          3
5DK+S(納戸)         3
3K+S(納戸)          2
4DK+S(納戸)         2
1LK+S(納戸)         2
6DK               2
1LK               1
6K                1
2R                1
11R               1
8LDK              1
Name: layout, dtype: int64

In [21]:
test[test['layout']=='11R']

,id,location,access,layout,age,direction,area,floor,bath_toilet,kitchen,...,23ku_countall,area_num_countall,age_countall,floor_countall,max_floor_countall,layout_countall,room_num_countall,direction_countall,facilities_countall,contact_period_countall
8483,39954,東京都世田谷区桜上水５丁目,京王線\t桜上水駅\t徒歩5分\t\t京王線\t上北沢駅\t徒歩10分\t\t東急世田谷線\...,11R,64年0ヶ月,南,16m2,1,専用バス／\t専用トイレ／\tシャワー,ガスコンロ／\t冷蔵庫あり,...,5971,262,1,12112,12311,1,47969,17696,54,45052


In [22]:
test[test['layout']=='2R']

,id,location,access,layout,age,direction,area,floor,bath_toilet,kitchen,...,23ku_countall,area_num_countall,age_countall,floor_countall,max_floor_countall,layout_countall,room_num_countall,direction_countall,facilities_countall,contact_period_countall
24546,56017,東京都北区志茂２丁目,京浜東北線\t赤羽駅\t徒歩10分\t\t南北線\t志茂駅\t徒歩4分\t\t南北線\t赤羽...,2R,55年10ヶ月,南,14.8m2,2,共同トイレ,NaN,...,2061,8,7,17202,12311,1,11021,17696,1,45052
